<a href="https://colab.research.google.com/github/matdjohnson-at-umass-dot-edu/CS532-FinalProject/blob/main/CS532_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ir-datasets

import ir_datasets
import xml.etree.ElementTree as ElementTree
from datetime import datetime
import re
import textwrap
import csv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18995 sha256=16da400171b0eb9004c7312e7f83d2c714eb7404185755513b6f875ef4930e6f
  Stored in directory: /root/.cache/pip/wheels/63/f9/dc/2dd16d3330e327236e4d407941975c42d5159d200cdb7922d8
  Created wheel for cbor: filename=cbor-1.0.0-cp311-cp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! export data_directory='/content/drive/MyDrive/CS532-FinalProject/WikipediaDump20250201-Unzipped'; \
#   for file in $(ls ${data_directory}); \
#     do echo "$(date +%Y-%m-%d-%T) - Unzipping file ${file} ..."; \
#     bzip2 -d ${data_directory}/${file}; \
#   done


# wikipedia_file_directory = "/content/drive/MyDrive/CS532-FinalProject/WikipediaDump20250201-Unzipped"
# sample_filename = "enwiki-20250201-pages-meta-current1.xml-p1p41242"
# wikipedia_sample_file = open(f"{wikipedia_file_directory}/{sample_filename}")
# print(f"{str(datetime.now())} - start parse of XML")
# wikipedia_xml_tree = ElementTree.parse(wikipedia_sample_file)
# print(f"{str(datetime.now())} - end parse of XML")
# print(f"{str(datetime.now())} - start read of XML root")
# wikipedia_xml_tree.findall(".")
# print(f"{str(datetime.now())} - end read of XML root")


# wikipedia_xml_root_children = wikipedia_xml_tree.findall('*')
# print(f"{len(wikipedia_xml_root_children)}")
# for i in range(0, 100):
#     print(f"element-{i}: {wikipedia_xml_root_children[i]}")


# namespace_prefix = '\{.*\}'
# map_of_tags_to_indices = dict()
#
# for i in range(0, len(wikipedia_xml_root_children)):
#     tag_for_element = re.sub(namespace_prefix, "", wikipedia_xml_root_children[i].tag)
#     list_for_tag = None
#     if map_of_tags_to_indices.get(tag_for_element) is None:
#         list_for_tag = list()
#         map_of_tags_to_indices[tag_for_element] = list_for_tag
#     else:
#         list_for_tag = map_of_tags_to_indices.get(tag_for_element)
#     list_for_tag.append(i)
#
# for k, v in map_of_tags_to_indices.items():
#     print(f"key={k} len(value)={len(v)}")
#
# print()
# ElementTree.indent(wikipedia_xml_root_children[0])
# print(ElementTree.tostring(wikipedia_xml_root_children[0]).decode())
# print()
# ElementTree.indent(wikipedia_xml_root_children[1])
# print(ElementTree.tostring(wikipedia_xml_root_children[1]).decode())


# namespace_prefix = '\{.*\}'
# map_of_tags_to_indices = dict()
#
# for i in range(0, len(wikipedia_xml_root_children)):
#     tag_for_element = re.sub(namespace_prefix, "", wikipedia_xml_root_children[i].tag)
#     list_for_tag = None
#     if map_of_tags_to_indices.get(tag_for_element) is None:
#         list_for_tag = list()
#         map_of_tags_to_indices[tag_for_element] = list_for_tag
#     else:
#         list_for_tag = map_of_tags_to_indices.get(tag_for_element)
#     list_for_tag.append(i)
#
# for k, v in map_of_tags_to_indices.items():
#     print(f"key={k} len(value)={len(v)}")
#
# print()
# ElementTree.indent(wikipedia_xml_root_children[0])
# print(ElementTree.tostring(wikipedia_xml_root_children[0]).decode())
# print()
# ElementTree.indent(wikipedia_xml_root_children[1])
# print(ElementTree.tostring(wikipedia_xml_root_children[1]).decode())


# tag_types = set(map_of_tags_to_indices.keys())
# print(tag_types)
# for element in wikipedia_xml_tree.iter():
#     tag_for_element = re.sub(namespace_prefix, "", element.tag)
#     if tag_for_element == 'mediawiki':
#         print()
#         ElementTree.indent(element)
#         print(ElementTree.tostring(element).decode())
#     # tag_for_element = element.tag
#     tag_types.add(tag_for_element)
# print(f"len(tag_types)={len(tag_types)}")


# tag_type_print_lines = textwrap.wrap(str(tag_types))
# for line in tag_type_print_lines:
#     print(line)


# for element in wikipedia_xml_root_children:
#     tag_for_element = re.sub(namespace_prefix, "", element.tag)

In [ ]:
dataset_directory = "/content/drive/MyDrive/CS532-FinalProject/ir_datasets"
dataset_name = "wikir/en1k/training"
dataset_filename = dataset_name.replace("/", "_")

train_dataset = ir_datasets.load("wikir/en1k/training")

docs_file = open(f"{dataset_directory}/{dataset_filename}", 'w+')
csv_writer = csv.writer(docs_file, dialect='unix')

for doc in train_dataset.docs_iter():
    csv_writer.writerow([doc.doc_id, doc.text])

docs_file.close()

In [ ]:
from pyspark import SparkContext, StorageLevel

if spark is not None:
    spark.stop()
spark = SparkContext()

train_rdd = spark.textFile(f"{dataset_directory}/{dataset_filename}")
train_rdd.persist(StorageLevel.MEMORY_AND_DISK)

def map_function(row):
    csv_file_line_elements = row.split('\",\"')
    doc_id = re.sub("[^A-Za-z0-9 ]", "", csv_file_line_elements[0])
    words_for_row = re.sub("[^A-Za-z0-9 ]", "", csv_file_line_elements[1]).lower().split(' ')
    return list(zip(words_for_row, list(doc_id for i in range(0, len(words_for_row)))))

train_rdd = train_rdd.flatMap(map_function)
